# imports

In [1]:
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds

# constants

In [2]:
BATCH_SIZE = 128
EPOCHS = 6
LEARNING_RATE = 0.001
NEURONS = 128

# fetch

In [3]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

df = tfds.as_dataframe(ds_test, ds_info)
df.head()

2025-03-20 20:14:58.676906: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:387] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608
2025-03-20 20:14:59.247856: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


,image,label
0,"[[[0], [0], [0], [0], [0], [0], [0], [0], [0],...",2
1,"[[[0], [0], [0], [0], [0], [0], [0], [0], [0],...",0
2,"[[[0], [0], [0], [0], [0], [0], [0], [0], [0],...",4
3,"[[[0], [0], [0], [0], [0], [0], [0], [0], [0],...",8
4,"[[[0], [0], [0], [0], [0], [0], [0], [0], [0],...",7


# prepare

In [4]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

# train

In [5]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(NEURONS, activation='relu'),
  tf.keras.layers.Dense(10, activation='sigmoid')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=EPOCHS,
    validation_data=ds_test,
)

Epoch 1/6


/Users/jtucker/Desktop/working2/keras-by-example/.environment/lib/python3.11/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.6256 - sparse_categorical_accuracy: 0.8320 - val_loss: 0.2006 - val_sparse_categorical_accuracy: 0.9413
Epoch 2/6
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1785 - sparse_categorical_accuracy: 0.9511 - val_loss: 0.1410 - val_sparse_categorical_accuracy: 0.9590
Epoch 3/6
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.1236 - sparse_categorical_accuracy: 0.9642 - val_loss: 0.1199 - val_sparse_categorical_accuracy: 0.9650
Epoch 4/6
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0960 - sparse_categorical_accuracy: 0.9733 - val_loss: 0.0977 - val_sparse_categorical_accuracy: 0.9719
Epoch 5/6
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0747 - sparse_categorical_accuracy: 0.9780 - val_loss: 0.0864 - val_sparse_categorical_accuracy: 0.9738
Epoch 6/6
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0587 - sparse_categorical_accuracy: 0.9836 - val_loss: 0.0852 - val_sparse_categorical_accuracy: 0.9734


# predict

In [6]:
predictions = model.predict(
    ds_test,
    batch_size=BATCH_SIZE,
    verbose=0,
)
predictions_df = pd.DataFrame(predictions)
predictions_df.head()

,0,1,2,3,4,5,6,7,8,9
0,0.007473,0.006699,0.999991,0.946132,0.145714,0.145369,9.512436e-02,0.006286,0.390860,0.009487
1,1.000000,0.000041,0.002691,0.000662,0.000052,0.545225,9.689220e-01,0.028663,0.176884,0.006700
2,0.000752,0.005479,0.040034,0.076713,0.999996,0.018741,1.689374e-03,0.891848,0.040898,0.967033
3,0.237111,0.402666,0.151527,0.564121,0.000886,0.271262,1.909901e-02,0.037851,0.998919,0.438010
4,0.056296,0.000056,0.451079,0.781091,0.007659,0.019796,3.682432e-07,0.999996,0.062364,0.504355
